In [1]:
import os
import json
import pandas as pd
import traceback


In [2]:
from langchain_openai import ChatOpenAI


In [12]:
import os
key=os.getenv('OPEN_API_KEY')

In [13]:
print(key)

None


In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
llm=ChatOpenAI(
    open_api_key=key,
    modei="gpt-4o-mini",
    temperature=0.5
)

c:\Users\user\mcqgen\venv\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: WARNING! open_api_key is not default parameter.
                open_api_key was transferred to model_kwargs.
                Please confirm that open_api_key is what you intended.
  if await self.run_code(code, result, async_=asy):
c:\Users\user\mcqgen\venv\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: WARNING! modei is not default parameter.
                modei was transferred to model_kwargs.
                Please confirm that modei is what you intended.
  if await self.run_code(code, result, async_=asy):


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable